In [ ]:
import os
import cv2
import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

with open('drive/MyDrive/dpns/coco-stuff/meta.json', 'r') as f:
    meta_data = json.load(f)

class_id_to_index = {cls['id']: idx for idx, cls in enumerate(meta_data['classes'])}
class_title_to_index = {cls['title']: idx for idx, cls in enumerate(meta_data['classes'])}
n_classes = len(meta_data['classes'])

In [ ]:
def dice_loss(y_true, y_pred, smooth=1e-4):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)

    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f)

    return 1 - (2. * intersection + smooth) / (union + smooth)

In [ ]:
def combo_loss(y_true, y_pred):
     return tf.keras.losses.CategoricalCrossentropy()(y_true, y_pred) + dice_loss(y_true, y_pred)

In [ ]:

from tensorflow.keras.models import load_model
custom_objects = {'combo_loss': combo_loss}

model = load_model('drive/MyDrive/dpns/u_net_model.keras', custom_objects=custom_objects)

In [ ]:
def generate_colormap(num_classes):
    np.random.seed(42)
    colormap = np.random.randint(0, 255, size=(num_classes, 3), dtype=np.uint8)
    return colormap

In [ ]:
def apply_colormap(mask, colormap):
    rgb_mask = colormap[mask]
    return rgb_mask

In [ ]:
colormap = generate_colormap(n_classes)

In [ ]:
def overlay_mask_on_image(image, mask, alpha=0.5):
    overlayed_image = cv2.addWeighted(image, 1 - alpha, mask, alpha, 0)
    return overlayed_image

In [ ]:
def predict_mask_threshold(img_path, model, colormap, threshold=0.5, alpha=0.5):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    img_input = img_resized / 255.0

    pred_mask_probs = model.predict(tf.expand_dims(img_input, axis=0))[0]
    confidence_mask = np.max(pred_mask_probs, axis=-1)
    pred_mask = np.argmax(pred_mask_probs, axis=-1)

    filtered_pred_mask = np.where(confidence_mask >= threshold, pred_mask, -1)

    pred_mask_rgb = apply_colormap(filtered_pred_mask, colormap)

    overlayed_image = overlay_mask_on_image(img_resized, pred_mask_rgb, alpha=alpha)

    return img_resized, pred_mask_rgb, overlayed_image


In [ ]:
def display_multiple_images(image_paths, model, colormap, thresholds, alpha=0.7):
    num_images = len(image_paths)
    plt.figure(figsize=(18, num_images * 6))

    for i, img_path in enumerate(image_paths):
        threshold = thresholds[i] if i < len(thresholds) else 0.2
        img_resized, pred_mask_rgb, overlayed_image = predict_mask_threshold(img_path, model, colormap, threshold, alpha)

        plt.subplot(num_images, 3, i * 3 + 1)
        plt.title('Input Image')
        plt.imshow(img_resized)
        plt.axis('off')

        plt.subplot(num_images, 3, i * 3 + 2)
        plt.title('Predicted Mask')
        plt.imshow(pred_mask_rgb)
        plt.axis('off')

        plt.subplot(num_images, 3, i * 3 + 3)
        plt.title('Image with Mask')
        plt.imshow(overlayed_image)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

image_paths = [] #image paths
thresholds = [] # optional image thresholds
display_multiple_images(image_paths, model, colormap, thresholds, alpha=0.65)
